## Install the package dependencies before running this notebook

In [80]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [81]:
import lstm_encoder_decoder

In [82]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy 
import pickle
from glob import glob
import matplotlib.pyplot as plt

"""
    number of trajectories in each city
    # austin --  train: 43041 test: 6325 
    # miami -- train: 55029 test:7971
    # pittsburgh -- train: 43544 test: 6361
    # dearborn -- train: 24465 test: 3671
    # washington-dc -- train: 25744 test: 3829
    # palo-alto -- train:  11993 test:1686

    trajectories sampled at 10HZ rate, input 5 seconds, output 6 seconds
    
"""

'\n    number of trajectories in each city\n    # austin --  train: 43041 test: 6325 \n    # miami -- train: 55029 test:7971\n    # pittsburgh -- train: 43544 test: 6361\n    # dearborn -- train: 24465 test: 3671\n    # washington-dc -- train: 25744 test: 3829\n    # palo-alto -- train:  11993 test:1686\n\n    trajectories sampled at 10HZ rate, input 5 seconds, output 6 seconds\n    \n'

## Create a Torch.Dataset class for the training dataset

In [83]:
from glob import glob
import pickle
import numpy as np

ROOT_PATH = "../../argo2/"

cities = ["austin", "miami", "pittsburgh", "dearborn", "washington-dc", "palo-alto"]
splits = ["train", "test"]

def get_city_trajectories(city="palo-alto", split="train", normalized=False):
    outputs = None
    
    if split == "train":
        f_in = ROOT_PATH + split + "/" + city + "_inputs"
        inputs = pickle.load(open(f_in, "rb"))
        n = len(inputs)
        inputs = np.asarray(inputs)[:int(n * 0.8)]
        
        f_out = ROOT_PATH + split + "/" + city + "_outputs"
        outputs = pickle.load(open(f_out, "rb"))
        outputs = np.asarray(outputs)[:int(n * 0.8)]
        
    elif split == 'val':
        f_in = ROOT_PATH + 'train' + "/" + city + "_inputs"
        inputs = pickle.load(open(f_in, "rb"))
        n = len(inputs)
        inputs = np.asarray(inputs)[int(n * 0.8):]
        
        f_out = ROOT_PATH + 'train' + "/" + city + "_outputs"
        outputs = pickle.load(open(f_out, "rb"))
        outputs = np.asarray(outputs)[int(n * 0.8):]
    
    elif split == 'test':
        f_in = ROOT_PATH + split + "/" + city + "_inputs"
        inputs = pickle.load(open(f_in, "rb"))
        n = len(inputs)
        inputs = np.asarray(inputs)
        
    else:
        print('\"split\" should be train, val, or test.')
        inputs = None

    return inputs, outputs

class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, city: str, split:str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.transform = transform
        self.inputs, self.outputs = get_city_trajectories(city=city, split=split, normalized=False)
        
        # centering
        num_inputs, input_seq_len = self.inputs.shape[:2]
        num_outputs, output_seq_len = self.outputs.shape[:2]
        center_input = (
            np.repeat(self.inputs[:, 0, :], input_seq_len, axis=0)
            .reshape(num_inputs, input_seq_len, 2)
        )
        center_output = (
            np.repeat(self.inputs[:, 0, :], output_seq_len, axis=0)
            .reshape(num_outputs, output_seq_len, 2)
        )
        self.inputs -= center_input
        self.outputs -= center_output
        self.inputs = torch.Tensor(self.inputs)
        self.outputs = torch.Tensor(self.outputs)
        # self.inputs = self.inputs.reshape(num_inputs, input_seq_len * 2)
        # self.outputs = self.outputs.reshape(num_outputs, output_seq_len * 2)
        
        # build velocities
        # self.input_velocities = np.hstack((np.zeros((num_inputs, 1, 2)), np.diff(self.inputs, axis=1)))
        # self.output_velocities = np.hstack((np.zeros((num_outputs, 1, 2)), np.diff(self.outputs, axis=1)))

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        data = (
            self.inputs[idx], 
            self.outputs[idx]# , 
            # self.input_velocities[idx], 
            # self.output_velocities[idx]
        )
        return data

In [84]:
train_dataset = ArgoverseDataset('austin', 'train')

In [85]:
train_dataset.__getitem__(0)[0].shape

torch.Size([50, 2])

In [94]:
X_train = train_dataset.inputs.transpose(1,0)
y_train = train_dataset.outputs.transpose(1,0)
X_train.shape, y_train.shape

(torch.Size([50, 34432, 2]), torch.Size([60, 34432, 2]))

In [93]:
X_train.transpose(1,0).shape

torch.Size([50, 34432, 2])

In [96]:
model = lstm_encoder_decoder.lstm_seq2seq(
    input_size = 2, 
    hidden_size = 15
)
loss = model.train_model(
    X_train, 
    y_train, 
    n_epochs = 50, 
    target_len = 60, 
    batch_size = 5, 
    training_prediction = 'mixed_teacher_forcing', 
    teacher_forcing_ratio = 0.6, 
    learning_rate = 0.01, 
    dynamic_tf = False
)

100%|████████████████████████████████████████████| 50/50 [1:08:40<00:00, 82.41s/it, loss=1415.490]


In [105]:
val_dataset = ArgoverseDataset('austin', 'val')
X_val = val_dataset.inputs.transpose(1,0)
y_val = val_dataset.outputs.transpose(1,0)
X_val.shape, y_val.shape

(torch.Size([50, 8609, 2]), torch.Size([60, 8609, 2]))

In [131]:
mse = 0
for n in range(X_val.shape[1]):
    mse += ((model.predict(X_val[:, n, :], 60) - y_val[:, n, :].numpy()) ** 2).sum()
np.sqrt(mse / X_val.shape[1])

495.8628164528739

### Altering hyperparameters (2)

In [152]:
model2 = lstm_encoder_decoder.lstm_seq2seq(
    input_size = 2, 
    hidden_size = 25
)
loss = model2.train_model(
    X_train, 
    y_train, 
    n_epochs = 50, 
    target_len = 60, 
    batch_size = 500, 
    training_prediction = 'mixed_teacher_forcing', 
    teacher_forcing_ratio = 0.6, 
    learning_rate = 0.0001, 
    dynamic_tf = False
)

100%|██████████████████████████████████████████████| 50/50 [10:29<00:00, 12.59s/it, loss=1538.736]


In [153]:
mse = 0
for n in range(X_val.shape[1]):
    mse += ((model2.predict(X_val[:, n, :], 60) - y_val[:, n, :].numpy()) ** 2).sum()
np.sqrt(mse / X_val.shape[1])

436.2561363955811

In [154]:
loss = model2.train_model(
    X_train, 
    y_train, 
    n_epochs = 250, 
    target_len = 60, 
    batch_size = 500, 
    training_prediction = 'mixed_teacher_forcing', 
    teacher_forcing_ratio = 0.6, 
    learning_rate = 0.0001, 
    dynamic_tf = False
)

100%|███████████████████████████████████████████| 250/250 [1:01:36<00:00, 14.79s/it, loss=442.136]


In [155]:
mse = 0
for n in range(X_val.shape[1]):
    mse += ((model2.predict(X_val[:, n, :], 60) - y_val[:, n, :].numpy()) ** 2).sum()
np.sqrt(mse / X_val.shape[1])

251.11387759824171

In [161]:
train_dataset_paloalto = ArgoverseDataset('palo-alto', 'train')
X_train_paloalto = train_dataset_paloalto.inputs.transpose(1,0)
y_train_paloalto = train_dataset_paloalto.outputs.transpose(1,0)

mse = 0
for n in range(X_train_paloalto.shape[1]):
    mse += ((model2.predict(X_train_paloalto[:, n, :], 60) - y_train_paloalto[:, n, :].numpy()) ** 2).sum()
np.sqrt(mse / X_train_paloalto.shape[1])

357.92513315444955

In [164]:
torch.save(model2, 'model2.pth')

In [ ]:
# want: `np.concatenate(output_position[0, :], output velocity[:, :]))` as output